In [ ]:
using Pkg

In [ ]:
Pkg.status()

In [ ]:
# Pkg.add(Pkg.PackageSpec(;name="DifferentialEquations", version="6.16.0"))

In [ ]:
Pkg.status()

In [1]:
using Turing

┌ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0]
└ @ Base loading.jl:1278


In [2]:
using DifferentialEquations 

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1278


In [3]:
using Plots

In [4]:
u0 = [1.0,1.0]
tspan = (0.0,10.0)
function multiplicative_noise!(du,u,p,t)
  x,y = u
  du[1] = p[5]*x
  du[2] = p[6]*y
end
p = [1.5,1.0,3.0,1.0,0.1,0.1]

function lotka_volterra!(du,u,p,t)
  x,y = u
  α,β,γ,δ = p
  du[1] = dx = α*x - β*x*y
  du[2] = dy = δ*x*y - γ*y
end


prob_sde = SDEProblem(lotka_volterra!,multiplicative_noise!,u0,tspan,p)

ensembleprob = EnsembleProblem(prob_sde)
data = solve(ensembleprob,SOSRI(),saveat=0.1,trajectories=1000)

Turing.setadbackend(:forwarddiff)
@model function fitlv(data, prob)
    σ ~ InverseGamma(2,3)
    α ~ truncated(Normal(1.3,0.5),0.5,2.5)
    β ~ truncated(Normal(1.2,0.25),0.5,2)
    γ ~ truncated(Normal(3.2,0.25),2.2,4.0)
    δ ~ truncated(Normal(1.2,0.25),0.5,2.0)
    ϕ1 ~ truncated(Normal(0.12,0.3),0.05,0.25)
    ϕ2 ~ truncated(Normal(0.12,0.3),0.05,0.25)
    p = [α,β,γ,δ,ϕ1,ϕ2]
    prob = remake(prob, p=p)
    predicted = solve(prob,SOSRI(),saveat=0.1)

    if predicted.retcode != :Success
        Turing.acclogp!(_varinfo, -Inf)
    end
    for j in 1:length(data)
        for i = 1:length(predicted)
            data[j][i] ~ MvNormal(predicted[i],σ)
        end
    end
end;

┌ Warning: you are using the internal variable `_varinfo`
└ @ DynamicPPL /Users/zeynepenkavi/.julia/packages/DynamicPPL/wf0dU/src/compiler.jl:171


In [5]:
EnsembleSummary(data)

EnsembleSolution Solution of length 101 with uType:
Float64

In [ ]:
model = fitlv(data, prob_sde)
# chain = sample(model, NUTS(0.25), 200, init_theta = [1.5,1.3,1.2,2.7,1.2,0.12,0.12])
chain = sample(model, NUTS(0.25), 100)
# plot(chain)

In [ ]:
@code_warntype model.f(
    Random.GLOBAL_RNG,
    model,
    Turing.VarInfo(model),
    Turing.SampleFromPrior(),
    Turing.DefaultContext(),
    model.args...,
)

In [ ]:
function lotka_volterra(du,u,p,t)
  x, y = u
  α, β, γ, δ  = p
  du[1] = (α - β*y)x 
  du[2] = (δ*x - γ)y 
end
p = [1.5, 1.0, 3.0, 1.0]
u0 = [1.0,1.0]
prob1 = ODEProblem(lotka_volterra,u0,(0.0,10.0),p)
sol = solve(prob1,Tsit5())




sol1 = solve(prob1,Tsit5(),saveat=0.1)
odedata = Array(sol1) + 0.8 * randn(size(Array(sol1)))
plot(sol1, alpha = 0.3, legend = false); scatter!(sol1.t, odedata')

# set AD backend
Turing.setadbackend(:forwarddiff)

# set priors
priors = [truncated(Normal(1.5,0.5),0.5,2.5), truncated(Normal(1.2,0.5),0,2), truncated(Normal(3.0,0.5),1,4),truncated(Normal(1.0,0.5),0,2)]

# type stable model

@model function fitlv2(data, prob1, priors, ::Type{T} = Float64) where {T} # data should be a Vector
    σ ~ InverseGamma(2, 3)

    p ~ arraydist(priors) 

    predicted::Vector{Vector{T}} = solve(prob1, Tsit5(); p=p, saveat=0.1).u

    for i = 1:length(predicted)
        data[:,i] ~ MvNormal(predicted[i], σ)
    end
end


# instantiation
model2= fitlv2(odedata, prob1, priors)

In [ ]:
chain2 = sample(model2, NUTS(0.25), 100)

In [ ]:
Pkg.add(Pkg.PackageSpec(;name="SciMLBase", version="1.6.0"))

In [ ]:
using Pkg

In [ ]:
using SciMLBase